# Day 3 – Running on Real Quantum Hardware

## Install Qiskit and IBM provider

In [ ]:
!pip install qiskit qiskit-ibm-runtime

In [ ]:
# Import libraries
from qiskit import QuantumCircuit, Aer, transpile, execute
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import QiskitRuntimeService
import json
import matplotlib.pyplot as plt


## Authenticate and load IBM Quantum account

In [ ]:

# Replace 'YOUR_API_TOKEN' with your actual IBM Quantum API token
QiskitRuntimeService.save_account(channel="ibm_quantum", token="YOUR_API_TOKEN", overwrite=True)
service = QiskitRuntimeService(channel="ibm_quantum")

# List available backends
backends = service.backends()
print("Available backends:")
for b in backends:
    print("-", b.name)


## Build a simple Grover’s algorithm circuit (1-qubit example)

In [ ]:

qc = QuantumCircuit(1, 1)

# Oracle (mark |1>)
qc.x(0)
qc.h(0)
qc.z(0)
qc.h(0)
qc.x(0)

# Diffusion
qc.h(0)
qc.z(0)
qc.h(0)

# Measure
qc.measure(0, 0)

qc.draw('mpl')


## Run on simulator

In [ ]:
sim_backend = Aer.get_backend('qasm_simulator')
sim_job = execute(qc, sim_backend, shots=1024)
sim_result = sim_job.result()
sim_counts = sim_result.get_counts(qc)
print("Simulator counts:", sim_counts)


## Run on real IBM Quantum backend

In [ ]:
# Run on real IBM Quantum backend
real_backend = service.backend("ibm_osaka")  # Replace with available backend
transpiled = transpile(qc, real_backend)

real_job = service.run(program_id="circuit-runner", inputs={"circuits": transpiled, "shots": 1024}, backend=real_backend)
real_result = real_job.result()
real_counts = real_result[0].data["counts"]

# Convert to JSON
real_counts_json = json.dumps(real_counts)
print("Real device counts (JSON):", real_counts_json)


## Run on real IBM Quantum backend

In [ ]:
# Compare simulator vs real device
print("Simulator:", sim_counts)
print("Real device:", real_counts)

plot_histogram([sim_counts, real_counts], legend=["Simulator", "Real Device"])
plt.show()
